In [1]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os
from pathlib import Path
import re
import pickle
# ver qtos o modelo apenas de ner acertaria
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize 
import torch
from transformers import BertTokenizer,BertForTokenClassification
import numpy as np
import json   
from importlib import reload  # Python 3.4+
import random
from model import BertForChunkClassification
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup

from torch.utils.data import DataLoader
from importlib import reload 
#from eval import predict
import eval
#import importlib
#importlib.reload(module)
from dataset import InputFeatures, load_and_cache_examples
import dataset
import functionsAval as f

In [11]:
f = reload(f)
reload(dataset)
reload(eval)

path_model=r'.\model-multilabel-io\model'

In [162]:
tags, tokens = f.predictBERTNER_multi_IO(['The HB24 mRNA was absent or present at low levels in normal B and T lymphocytes ; however , with the appropriate activation signal HB24 mRNA was induced within several hours even in the presence of cycloheximide .'], path_model)
print('-----------')
print(tags)
print(tokens)

idx2tag: {0: 'RNA', 1: 'protein-protein', 2: 'RNA-cell_type', 3: 'protein-cell_line', 4: 'DNA-RNA', 5: 'cell_line-cell_line', 6: 'RNA-RNA', 7: 'cell_type-cell_type', 8: 'protein', 9: 'protein-DNA', 10: 'DNA-cell_line', 11: 'cell_line-RNA', 12: 'protein-cell_type', 13: 'O', 14: 'cell_line-DNA', 15: 'cell_type', 16: 'DNA', 17: 'cell_type-DNA', 18: 'cell_line', 19: 'DNA-DNA', 20: 'protein-RNA', 21: 'cell_type-cell_line', 22: '<pad>'}
-----------
[['O', 'DNA-RNA', 'RNA', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'DNA-RNA', 'RNA', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
[['the', 'hb24', 'mrna', 'was', 'absent', 'or', 'present', 'at', 'low', 'levels', 'in', 'normal', 'b', 'and', 't', 'lymphocytes', ';', 'however', ',', 'with', 'the', 'appropriate', 'activation', 'signal', 'hb24', 'mrna', 'was', 'induced', 'within', 'several', 'hours', 'even', 'in', 'the', 'presence', 'of', 'cycloheximide', '.']]


In [109]:

def save_obj(name, obj):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    with open('../obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    try:
        with open('../obj/' + name + '.pkl', 'rb') as f:
            return pickle.load(f)
    except:
        print('ERROO')
        raise
        return None
#save_obj('dicEntidadesTesteGenia', dicEntidades)

def loadSentencesTest():
    # gabarito = dicSentences_new.pkl
    #dicSentences_train = load_obj('dic_sentencesTrain.pkl')
    #dicSentences_train[32]
    # gabarito = dicSentences_new.pkl
    print('Pegando sentencas de teste gabarito: dic_sentencesTest.pkl')
    dicSentences_new_test = load_obj('dicEntidadesTesteGenia')
    return dicSentences_new_test

#dicSentences_new_test = loadSentencesTest()
#dicSentences_new_test


In [110]:
dicSentences_new_test

{0: [' There is a single methionine codon - initiated open reading frame of 1 , 458 nt in frame with a homeobox and a CAX repeat , and the open reading frame is predicted to encode a protein of 51 , 659 daltons .',
  [[['methionine codon - initiated open reading frame',
     'DNA',
     [4, 5, 6, 7, 8, 9, 10]],
    ['homeobox', 'DNA', [20]],
    ['CAX repeat', 'DNA', [23, 24]],
    ['open reading frame', 'DNA', [28, 29, 30]]]]],
 1: [' When the homeodomain from HB24 was compared to known mammalian and Drosophila homeodomains it was found to be only moderately conserved , but when it was compared to a highly diverged Drosophila homeodomain , H2 . 0 , it was found to be 80 % identical .',
  [[['homeodomain', 'DNA', [2]],
    ['HB24', 'DNA', [4]],
    ['homeodomain', 'DNA', [32]]],
   [['Drosophila homeodomain', 'DNA', [31, 32]]]]],
 2: [' The HB24 mRNA was absent or present at low levels in normal B and T lymphocytes ; however , with the appropriate activation signal HB24 mRNA was induce

In [128]:
BATCH=100
dicSentences_new_test = loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<BATCH}
print(dicSentences_new_test[0])
#print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))

sentences=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        #tokens = [tok[0] for tok in tokens]
        #sentences.append(' '.join(tokens).strip())
        sentences.append(tokens)

Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
1854
[' There is a single methionine codon - initiated open reading frame of 1 , 458 nt in frame with a homeobox and a CAX repeat , and the open reading frame is predicted to encode a protein of 51 , 659 daltons .', [[['methionine codon - initiated open reading frame', [4, 5, 6, 7, 8, 9, 10], 'DNA'], ['homeobox', [20], 'DNA'], ['CAX repeat', [23, 24], 'DNA'], ['open reading frame', [28, 29, 30], 'DNA']]]]
numero de sentencas no total: 100


In [163]:
sentences[:2]

[' There is a single methionine codon - initiated open reading frame of 1 , 458 nt in frame with a homeobox and a CAX repeat , and the open reading frame is predicted to encode a protein of 51 , 659 daltons .',
 ' When the homeodomain from HB24 was compared to known mammalian and Drosophila homeodomains it was found to be only moderately conserved , but when it was compared to a highly diverged Drosophila homeodomain , H2 . 0 , it was found to be 80 % identical .']

In [130]:
dicSentences_new_test

{0: [' There is a single methionine codon - initiated open reading frame of 1 , 458 nt in frame with a homeobox and a CAX repeat , and the open reading frame is predicted to encode a protein of 51 , 659 daltons .',
  [[['methionine codon - initiated open reading frame',
     [4, 5, 6, 7, 8, 9, 10],
     'DNA'],
    ['homeobox', [20], 'DNA'],
    ['CAX repeat', [23, 24], 'DNA'],
    ['open reading frame', [28, 29, 30], 'DNA']]]],
 1: [' When the homeodomain from HB24 was compared to known mammalian and Drosophila homeodomains it was found to be only moderately conserved , but when it was compared to a highly diverged Drosophila homeodomain , H2 . 0 , it was found to be 80 % identical .',
  [[['homeodomain', [2], 'DNA'],
    ['HB24', [4], 'DNA'],
    ['homeodomain', [32], 'DNA']],
   [['Drosophila homeodomain', [31, 32], 'DNA']]]],
 2: [' The HB24 mRNA was absent or present at low levels in normal B and T lymphocytes ; however , with the appropriate activation signal HB24 mRNA was induce

In [164]:
print(tags)
print(tokens)

[['O', 'DNA-RNA', 'RNA', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'DNA-RNA', 'RNA', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
[['the', 'hb24', 'mrna', 'was', 'absent', 'or', 'present', 'at', 'low', 'levels', 'in', 'normal', 'b', 'and', 't', 'lymphocytes', ';', 'however', ',', 'with', 'the', 'appropriate', 'activation', 'signal', 'hb24', 'mrna', 'was', 'induced', 'within', 'several', 'hours', 'even', 'in', 'the', 'presence', 'of', 'cycloheximide', '.']]


In [182]:
def getDicPredictions(tags, tokens):
    listaTags=['DNA', 'RNA','cell_type', 'cell_line','protein']

    dic_predictions = {}
    i=0
    for tags_sentence, tokens_sentence in zip(tags, tokens):
        listaEntidades=[]
        isEnt=0
        labelAnterior=''
        entidade=''
        label=''
        #listaTags
        for classe in listaTags:
            for tag, token in zip(tags_sentence, tokens_sentence):
                #print(token, tag)
                labels = tag.split('-')
                if tag!='O' and classe in labels:
                    isEnt=1
                    #print('---label-->:', label)
                    #print('---labelAnterior-->:', labelAnterior)
                    #if label == labelAnterior:
                    if labelAnterior in classe:
                        # continuacao
                        entidade = entidade+' '+token
                    elif labelAnterior=='': # primeiro token da entidade
                        entidade=token
                        labelAnterior=classe
                    else: # mudou de entidade
                        listaEntidades.append([entidade, labelAnterior])
                        entidade=token
                    labelAnterior = classe
                else:
                    if isEnt==1:
                        if entidade:
                            #print('---label-->:', label)
                            listaEntidades.append([entidade, classe])
                    entidade=''
                    labelAnterior=''
                    isEnt=0
                    label=''

        if len(listaEntidades)>0:
            dic_predictions[i]=['FRASE1', listaEntidades]
        i=i+1

    return dic_predictions

In [183]:
dic_predictions

{0: [[['there', 0],
   ['is', 1],
   ['a', 2],
   ['single', 3],
   ['methionine', 4],
   ['codon', 5],
   ['-', 6],
   ['initiated', 7],
   ['open', 8],
   ['reading', 9],
   ['frame', 10],
   ['of', 11],
   ['1', 12],
   [',', 13],
   ['458', 14],
   ['nt', 15],
   ['in', 16],
   ['frame', 17],
   ['with', 18],
   ['a', 19],
   ['homeobox', 20],
   ['and', 21],
   ['a', 22],
   ['cax', 23],
   ['repeat', 24],
   [',', 25],
   ['and', 26],
   ['the', 27],
   ['open', 28],
   ['reading', 29],
   ['frame', 30],
   ['is', 31],
   ['predicted', 32],
   ['to', 33],
   ['encode', 34],
   ['a', 35],
   ['protein', 36],
   ['of', 37],
   ['51', 38],
   [',', 39],
   ['659', 40],
   ['daltons', 41],
   ['.', 42]],
  [['methionine codon - initiated open reading frame',
    [4, 5, 6, 7, 8, 9, 10],
    'DNA'],
   ['homeobox', [20], 'DNA'],
   ['cax repeat', [23, 24], 'DNA'],
   ['open reading frame', [28, 29, 30], 'DNA']]],
 1: [[['when', 0],
   ['the', 1],
   ['homeodomain', 2],
   ['from', 3],


In [184]:
tags, tokens = f.predictBERTNER_multi_IO(sentences, path_model)
dic_predictions = getDicPredictions(tags, tokens)
print(dic_predictions[0])
print('len(dic_predictions):', len(dic_predictions))

idx2tag: {0: 'RNA', 1: 'protein-protein', 2: 'RNA-cell_type', 3: 'protein-cell_line', 4: 'DNA-RNA', 5: 'cell_line-cell_line', 6: 'RNA-RNA', 7: 'cell_type-cell_type', 8: 'protein', 9: 'protein-DNA', 10: 'DNA-cell_line', 11: 'cell_line-RNA', 12: 'protein-cell_type', 13: 'O', 14: 'cell_line-DNA', 15: 'cell_type', 16: 'DNA', 17: 'cell_type-DNA', 18: 'cell_line', 19: 'DNA-DNA', 20: 'protein-RNA', 21: 'cell_type-cell_line', 22: '<pad>'}
['FRASE1', [[' methionine codon - initiated open reading frame', 'DNA'], [' homeobox', 'DNA'], [' cax repeat', 'DNA'], [' open reading frame', 'DNA']]]
len(dic_predictions): 94


In [185]:
dic_predictions[2]

['FRASE1',
 [[' hb24', 'DNA'],
  [' hb24', 'DNA'],
  [' hb24 mrna', 'RNA'],
  [' hb24 mrna', 'RNA']]]

In [186]:
dicSentences_new_test[2]

[' The HB24 mRNA was absent or present at low levels in normal B and T lymphocytes ; however , with the appropriate activation signal HB24 mRNA was induced within several hours even in the presence of cycloheximide .',
 [[['HB24 mRNA', [1, 2], 'RNA'], ['HB24 mRNA', [24, 25], 'RNA']],
  [['T lymphocytes', [14, 15], 'cell_type']]]]

In [176]:
def AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH):

    region_true_list, region_pred_list = list(), list() # labels
    region_true_count, region_pred_count = 0, 0 # contagem
    numErro1=0
    numErro2=0

    for i in range(0, BATCH, 1):
        if i<len(dicSentences_new_test):
            #print('\n---Label vs predicao--------')
            #print('frase:', dicSentences_new_test[i][0])
            #print('i:', i)
            #print(dicSentences_new_test[i][1])
            #print(dic_predictions_all[i][1])


            entidades_gabarito = dicSentences_new_test[i][1]
            try:
                entidades_preditas = dic_predictions_all[i][1]
            except:
                print('erro, caiu no except na AvalFinal:', str(i))
            #print('---entidades_gabarito--:', entidades_gabarito)
            #print('entidades_preditas:', entidades_preditas)
            for entidade_gabarito in entidades_gabarito:
                #print('---entidade_gabarito[0]--:', entidade_gabarito[0])
                entidade_gabarito=entidade_gabarito[0]
                indices_gabarito = entidade_gabarito[1]
                tag_gabarito = entidade_gabarito[2]
                region_true_count=region_true_count+1
                region_true_list.append(tag_gabarito)
                # ver se previu essa entidade
                previu=0
                for entidade_predita in entidades_preditas:
                    indices_predita = entidade_predita[1]
                    tag_predita = entidade_predita[2]
                    if indices_predita == indices_gabarito:
                        region_pred_list.append(tag_predita)
                        previu=1
                        if tag_predita !='O':
                            region_pred_count=region_pred_count+1
                        break
                if previu==0:
                    numErro1=numErro1+1
                    region_pred_list.append('O')

            # agora o contrario, ver o q previu mas nao era

            for entidade_predita in entidades_preditas:
                indices_predita = entidade_predita[1]
                tag_predita = entidade_predita[2]
                # ver se a entidade prevista existe ou é FP
                existe=0
                for entidade_gabarito in entidades_gabarito:
                    entidade_gabarito=entidade_gabarito[0]
                    indices_gabarito = entidade_gabarito[1]
                    if indices_predita == indices_gabarito:
                        existe=1
                        break
                if existe==0:
                    numErro2 = numErro2+1
                    region_true_list.append('O')
                    region_pred_list.append(tag_predita)

    #print('===resultados====')
    #print('region_true_list:', region_true_list)
    #print('region_pred_list:', region_pred_list)

    print('numErro1:', numErro1)
    print('numErro2:', numErro2)
    #print(classification_report(region_true_list, region_pred_list, labels=['O', 'PROTEIN', 'DNA', 'RNA', 'CELL_TYPE', 'CELL_LINE'], target_names=['O', 'PROTEIN', 'DNA', 'RNA', 'CELL_TYPE', 'CELL_LINE'], digits=6))
    print(classification_report(region_true_list, region_pred_list, digits=6))

    return region_true_list, region_pred_list



In [177]:
region_true_list[:10]

['DNA', 'O', 'O', 'O', 'DNA', 'DNA', 'O', 'O', 'RNA', 'cell_type']

In [178]:
region_pred_list[:10]

['DNA', 'DNA', 'DNA', 'DNA', 'DNA', 'DNA', 'DNA', 'DNA', 'O', 'O']

In [179]:
dicSentences_new_test[2][1]

[[['HB24 mRNA', [1, 2], 'RNA'], ['HB24 mRNA', [24, 25], 'RNA']],
 [['T lymphocytes', [14, 15], 'cell_type']]]

In [180]:
dic_predictions[2][1]

[['hb24', [1], 'DNA-RNA'],
 ['mrna', [2], 'RNA'],
 ['hb24', [24], 'DNA-RNA'],
 ['mrna', [25], 'RNA']]

In [181]:
region_true_list, region_pred_list = AvalFinal(dicSentences_new_test, dic_predictions, BATCH)
print(confusion_matrix(region_true_list, region_pred_list))

numErro1: 53
numErro2: 172
                 precision    recall  f1-score   support

            DNA   0.363636  0.600000  0.452830        60
        DNA-RNA   0.000000  0.000000  0.000000         0
              O   0.000000  0.000000  0.000000       172
            RNA   0.250000  0.333333  0.285714         3
      cell_line   0.464286  0.619048  0.530612        21
      cell_type   0.789474  0.789474  0.789474        19
        protein   0.353846  0.676471  0.464646        68
    protein-DNA   0.000000  0.000000  0.000000         0
protein-protein   0.000000  0.000000  0.000000         0

       accuracy                       0.323615       343
      macro avg   0.246805  0.335369  0.280364       343
   weighted avg   0.208104  0.323615  0.250046       343

[[36  0 20  0  0  0  4  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [63  2  0  3 15  4 80  2  3]
 [ 0  0  2  1  0  0  0  0  0]
 [ 0  0  8  0 13  0  0  0  0]
 [ 0  0  4  0  0 15  0  0  0]
 [ 0  0 19  0  0  0 46  2  1]
 [ 0  0  0  0  0  0